### Approaches to Solving MDPs I: Dynamic Programming

Last updated: March 15, 2022


### SOURCES 

- Reinforcement Learning, RS Sutton & AG Barto, 2nd edition. Chapter 4
- Mastering Reinforcement Learning with Python, Enes Bilgin. Chapter 5

### LEARNING OUTCOMES

- Understand the conceptual ideas behind dynamic programming 
-  Study the Bellman optimality equations for estimating the optimal value functions
- Discuss the strengths and limitations of Dynamic Programming

### CONCEPTS

- Dynamic Programming
- Policy Iteration
- Bellman optimality equations


#### There are three important approaches to solving MDPs:

- Dynamic Programming (DP)
- Monte Carlo Simulation (MC)
- Temporal Difference (TD)

**DP methods**  
Provide optimal solutions to Markov decision processes (MDPs)  
Much more efficient than exhaustive search  
Developed by Richard Bellman in the 1950s  

Strengths: 

- Closed-form equations for optimality
- Intuitive, as it exploits the property of overlapping substructure
- Efficient computation

Limitations:

- Requires complete knowledge of the state transition and reward dynamics of the environment. This is often unrealistic.
- When the state space is continuous or massive, the required runtime becomes infeasible

**MC methods**  
Use sampled transitions from the environment, so transition matrix (dynamics of system) is not required

Relies on convergence properties when measuring value functions

**TD methods**  

In this notebook we will concentrate on Dynamic Programming.

---  

#### Motivating the Dynamic Programming Method

We will only briefly touch on DP: it is very instructive but generally impractical for large-scale problems.

Consider this example where a robot needs to decide the best path to traverse on a binary tree.  

This tree has three layers (ignoring the start node).
Rewards are located on each node.

We make this problem a little easier with these dynamics:
- discount factor will be $\gamma=1$ (no discounting)
- the robot decides which path to take (the action) with probability 1
- the environment is deterministic: the robot will take the decided upon path

Task: select the path that maximizes reward. What is the path and what is the maximum reward?  

**Example 1: Robot maximization task**

![tree_task1](images/tree_task1_binary.png)

Working from left to right is a brute force method that solves the problem. Is there a more efficient way?



Hint: Think about working backwards from right to left, backing the rewards to earlier layers in the tree.

[enter your ideas here] 

Starting at the next-to-last layer, it will look ahead from each node to the future rewards.

It will select the next step with highest reward, acting greedily.  It can compute this maximum before taking a step.


![tree_task1](images/tree_task1_working_backwards_binary.png)

For example at the top node with reward 2, it would select the path leading to 3.  
The value of the top node with reward 2 is then 2 + 3 = 5. It would receive immediate reward 2 and would receive 3 at the next step.

Let's back up the rewards earlier in the tree and update the values at layer 2.

![tree_task1](images/tree_task1_working_backwards_adding_rewards1_binary.png)

Notice the structure: use the potential values in the next step $t+1$ to compute the value at the earlier step $t$

This is iterative and can be easily coded for massive trees

In general, **problems that have overlapping substructure are good candidates for dynamic programming solutions**

Repeat this procedure, working backwards through the tree and pushing values earlier

![tree_task1](images/tree_task1_working_backwards_adding_rewards2_binary.png)

![tree_task1](images/tree_task1_working_backwards2_adding_rewards2_binary.png)

This produces the best path and maximum reward of 11.

**Notice:** This dynamic programming approach is much more efficient than brute force search. Can you see why?

Answer: DP requires computing the value of each node only once, since values are pushed backwards.  

**Exercise 1**  

Write an algorithm that will apply the data and compute the maximum value for the robot problem.  Print the results to verify it works properly.

In [52]:
# Solution
# set up a structure to hold nodes and rewards. 

import numpy as np
rewards = np.array([
                    [1,2,1],
                    [3,1,3],
                    [0,3,4],
                    [0,0,5]
                  ])

In [53]:
max_depth = rewards.shape[0]-1 

# work backwards across layers
for layer in [2,1]:
    # work top down with each layer
    for depth in range(max_depth):
        # compute max over child paths and add to prior layer (time step)
        rewards[depth,layer-1] += max(rewards[depth,layer],rewards[depth+1,layer])

print(rewards, '\n')
print('max reward:', max(rewards[:,0]))

[[ 6  5  1]
 [11  5  3]
 [ 8  8  4]
 [ 0  0  5]] 

max reward: 11


**Relationship to Pricing an American Call Option**  

If you have priced options on a binomal lattice, you will notice similar ideas being applied.  

A *call option* on a stock allows the holder to buy the stock (*exercise* the option) for a fixed price (*strike price*) by a fixed date (*expiry*).    
An *American* call option allows the holder to exercise before expiry.  He will take the action - *(exercise, no exercise)* - that maximizes value.  

For the option pricing problem, it is a bit more involved than the robot problem:

- there will be discounting to reflect time value of money
- the environment is stochastic: the stock price will fluctuate with a random component

---

#### Key Formulas in Dynamic Programming

The formulas will handle the general case to include discounting, a policy that follows a distribution and uncertainty in the environment.

Here are the Bellman optimality equations for computing optimal state value $v_*(s)$ and optimal action value $q_*(s,a)$

$v_*(s) = \underset{a}{\operatorname{\max}} E[R_{t+1} + \gamma v_*(S_{t+1}) | S_t=s]$


$q_*(s,a) = E[R_{t+1} + \gamma \underset{a}{\operatorname{\max}} q_*(S_{t+1},a) | S_t=s, A_t=a]$

For $v_*(s)$, the agent selects actions that maximize the state value for each state *s*.

For $q_*(s,a)$, the agent selects action *a* from state *s* and follows the optimal policy afterwards. 

In order to compute expectations, the transition probabilities need to be known in advance.

Additionally, the expectation calculation grows with the number of states.

#### Policy Evaluation and Improvement

An important goal is to discover the optimal policy.  
This can be done by measuring the value of the current policy, trying to improve the policy, and repeating these two steps.

**Policy iteration** consists of two simultaneous, interacting processes: 
- *policy evaluation*: make the value function consistent with the current policy 
- *policy improvement*: make the policy greedy given the current value function

The graphic below demonstrates this idea, using iteration to build a better policy and then value the updated policy.

![gpi](images/gpi.png)

Algorithms for policy evaluation and value iteration can be found in Sutton and Barto, for example.

Since we often won't have the transition probabilities to solve real-world problems, we won't discuss them here.

Next, we will study a technique that sidesteps the need for transition probabilities: Monte Carlo simulation.

---